In [1]:
import pandas as pd
from meta_project.data.data_loader import DataLoader
data_loader = DataLoader()

df = data_loader.load_and_merge_data()
df.head()

2025-04-08 22:48:28.868 | INFO     | meta_project.config:<module>:11 - PROJ_ROOT path is: C:\Users\Michal\Desktop\MAML\metaLearningCUB-200-2011


,id,image_name,image_id,class_id,class_name,is_training_image
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,1,001.Black_footed_Albatross,0
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...,2,1,001.Black_footed_Albatross,1
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...,3,1,001.Black_footed_Albatross,0
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...,4,1,001.Black_footed_Albatross,1
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...,5,1,001.Black_footed_Albatross,1


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from meta_project.data.data_loader import DataLoader as MetaDataLoader
from torchvision import transforms
import random
from PIL import Image
import os
import copy

# --- Configuration ---
N_WAY = 5
K_SHOT = 1
N_QUERY = 15
N_TASKS_PER_EPOCH = 1000
BATCH_SIZE = 16
LR_INNER = 0.01
LR_OUTER = 0.001
NUM_INNER_STEPS = 1
IMAGE_RESIZE = 84
IMAGE_DIR = 'data/raw/CUB_200_2011/images'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Model Definition (Adapted for MAML) ---
class SimpleCNN(nn.Module):
    def __init__(self, n_way):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2) # 84x84 -> 42x42

        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2) # 42x42 -> 21x21

        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2) # 21x21 -> 10x10

        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(2) # 10x10 -> 5x5

        flattened_size = 64 * 5 * 5
        self.fc1 = nn.Linear(flattened_size, n_way)

    def forward(self, x):
        x = self.pool1(self.relu1(self.bn1(self.conv1(x))))
        x = self.pool2(self.relu2(self.bn2(self.conv2(x))))
        x = self.pool3(self.relu3(self.bn3(self.conv3(x))))
        x = self.pool4(self.relu4(self.bn4(self.conv4(x))))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

# --- MAML Implementation (FOMAML using deepcopy) ---
class MAML(nn.Module):
    def __init__(self, model, lr_inner=0.01, lr_outer=0.001, num_inner_steps=1):
        super(MAML, self).__init__()
        self.model = model
        self.lr_inner = lr_inner
        self.lr_outer = lr_outer
        self.num_inner_steps = num_inner_steps
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr_outer)
        self.loss_fn = nn.CrossEntropyLoss()

        self.model.to(device)

    def forward(self, x):
        return self.model(x.to(device))

    def meta_train_batch(self, task_batch):
        batch_x_train, batch_y_train, batch_x_test, batch_y_test = task_batch
        num_tasks_in_batch = batch_x_train.size(0)

        total_outer_loss = torch.tensor(0.0, device=device)

        self.optimizer.zero_grad()

        for task_idx in range(num_tasks_in_batch):
            x_train = batch_x_train[task_idx]
            y_train = batch_y_train[task_idx]
            x_test = batch_x_test[task_idx]
            y_test = batch_y_test[task_idx]

            x_train, y_train = x_train.to(device), y_train.to(device)
            x_test, y_test = x_test.to(device), y_test.to(device)

            # --- Inner Loop Adaptation ---
            temp_model = copy.deepcopy(self.model)
            temp_model.train()
            inner_optimizer = optim.SGD(temp_model.parameters(), lr=self.lr_inner)

            for _ in range(self.num_inner_steps):
                inner_optimizer.zero_grad()
                y_pred_support = temp_model(x_train)
                loss_support = self.loss_fn(y_pred_support, y_train)
                loss_support.backward()
                inner_optimizer.step()

            temp_model.eval()

            y_pred_query = temp_model(x_test)
            loss_query = self.loss_fn(y_pred_query, y_test)

            total_outer_loss = total_outer_loss + loss_query

        # --- Outer Loop Update ---
        average_outer_loss = total_outer_loss / num_tasks_in_batch

        average_outer_loss.backward()

        self.optimizer.step()

        return average_outer_loss.item()

# --- Meta Dataset ---
class MetaDataset(Dataset):
    def __init__(self, df, n_way, k_shot, n_query, image_dir, transform, num_tasks):
        self.df = df
        self.n_way = n_way
        self.k_shot = k_shot
        self.n_query = n_query
        self.image_dir = image_dir
        self.transform = transform
        self.num_tasks = num_tasks

        self.classes = df['class_name'].unique().tolist()
        self.class_to_images = {cls: df[df['class_name'] == cls]['image_name'].tolist()
                                for cls in self.classes}

        if len(self.classes) < n_way:
            raise ValueError(f"Dataset has only {len(self.classes)} classes, but n_way={n_way} requested.")

    def __len__(self):
        return self.num_tasks

    def __getitem__(self, index):
        sampled_classes = random.sample(self.classes, self.n_way)

        task_support_images, task_support_labels = [], []
        task_query_images, task_query_labels = [], []

        task_class_map = {cls_name: i for i, cls_name in enumerate(sampled_classes)}

        for cls_name in sampled_classes:
            available_images = self.class_to_images[cls_name]

            required_images = self.k_shot + self.n_query
            if len(available_images) < required_images:
                print(f"Warning: Class {cls_name} has only {len(available_images)} images, sampling with replacement.")
                selected_indices = random.choices(range(len(available_images)), k=required_images)
                selected_images = [available_images[i] for i in selected_indices]
            else:
                selected_images = random.sample(available_images, required_images)

            task_label = task_class_map[cls_name]

            support_img_names = selected_images[:self.k_shot]
            query_img_names = selected_images[self.k_shot:]

            try:
                support_imgs = [self.transform(Image.open(os.path.join(self.image_dir, img)).convert('RGB')) for img in support_img_names]
                query_imgs = [self.transform(Image.open(os.path.join(self.image_dir, img)).convert('RGB')) for img in query_img_names]
            except FileNotFoundError as e:
                print(f"Error loading image: {e}. Check IMAGE_DIR and image paths.")

            task_support_images.extend(support_imgs)
            task_support_labels.extend([task_label] * self.k_shot)
            task_query_images.extend(query_imgs)
            task_query_labels.extend([task_label] * self.n_query)

        x_train = torch.stack(task_support_images)
        y_train = torch.tensor(task_support_labels, dtype=torch.long)
        x_test = torch.stack(task_query_images)
        y_test = torch.tensor(task_query_labels, dtype=torch.long)

        perm_support = torch.randperm(len(x_train))
        x_train = x_train[perm_support]
        y_train = y_train[perm_support]

        perm_query = torch.randperm(len(x_test))
        x_test = x_test[perm_query]
        y_test = y_test[perm_query]


        return x_train, y_train, x_test, y_test

# --- Data Loading ---
transform = transforms.Compose([
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.Resize((IMAGE_RESIZE, IMAGE_RESIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Loading data...")
try:
    data_loader_meta = MetaDataLoader(base_path='.')
    df = data_loader_meta.load_and_merge_data()
    print(f"Loaded DataFrame with {len(df)} entries.")
    print(f"Image directory used: {os.path.abspath(IMAGE_DIR)}")
except Exception as e:
    print(f"Error loading data using MetaDataLoader: {e}")
    print("Please ensure MetaDataLoader is correctly implemented and paths are set.")
    exit()


meta_dataset = MetaDataset(df, n_way=N_WAY, k_shot=K_SHOT, n_query=N_QUERY,
                           image_dir=IMAGE_DIR, transform=transform, num_tasks=N_TASKS_PER_EPOCH)

meta_dataloader = DataLoader(meta_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)


# --- Initialization and Training Loop ---
print("Initializing model...")
model = SimpleCNN(n_way=N_WAY)
maml = MAML(model, lr_inner=LR_INNER, lr_outer=LR_OUTER, num_inner_steps=NUM_INNER_STEPS)

print("Starting meta-training...")
num_epochs = 100
for epoch in range(num_epochs):
    maml.model.train()
    epoch_loss = 0.0
    task_count = 0

    for i, task_batch in enumerate(meta_dataloader):
        loss = maml.meta_train_batch(task_batch)

        if loss is not None:
            epoch_loss += loss
            task_count += len(task_batch)

        if (i + 1) % 10 == 0:
             print(f"  Batch {i+1}/{len(meta_dataloader)}, Approx Loss: {loss:.4f}")

    avg_epoch_loss = epoch_loss / (i+1)
    print(f"Epoch {epoch+1}/{num_epochs}: Average Approx Loss: {avg_epoch_loss:.4f}")

print("Meta-training finished.")

Using device: cuda
Loading data...
Error loading data using MetaDataLoader: DataLoader.__init__() got an unexpected keyword argument 'base_path'
Please ensure MetaDataLoader is correctly implemented and paths are set.
Initializing model...
Starting meta-training...
  Batch 50/63, Approx Loss: 1.6060
Epoch 1/100: Average Approx Loss: 1.6072
  Batch 50/63, Approx Loss: 1.6071


KeyboardInterrupt: 

: 